<a href="https://colab.research.google.com/github/HusnaRiyaz/Insilico-Molecular-Properties-of-Coumarin-using-DeepChem/blob/main/MyFinal_Solubility_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3501  100  3501    0     0  31258      0 --:--:-- --:--:-- --:--:-- 31258


add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.10
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [ ]:
!pip install --pre deepchem

     |████████████████████████████████| 573kB 24.5MB/s 


In [ ]:
import deepchem as dc
import numpy as np
import pandas as pd

In [ ]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='Weave')
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
print(train_dataset)

<DiskDataset X.shape: (902,), y.shape: (902, 1), w.shape: (902, 1), ids: ['CC(C)=CCCC(C)=CC(=O)' 'CCCC=C' 'CCCCCCCCCCCCCC' ...
 'Nc2cccc3nc1ccccc1cc23 ' 'C1CCCCCC1' 'OC1CCCCCC1'], task_names: ['measured log solubility in mols per litre']>


In [ ]:
print(valid_dataset)

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['Nc1ncnc2nc[nH]c12 ' 'Nc1nc(O)nc2nc[nH]c12 '
 'Fc1cccc(F)c1C(=O)NC(=O)Nc2cc(Cl)c(F)c(Cl)c2F ' ...
 'OC(Cn1cncn1)(Cn2cncn2)c3ccc(F)cc3F '
 'FC(F)(F)c1cccc(c1)N2CC(CCl)C(Cl)C2=O' 'CC1(C)CON(Cc2ccccc2Cl)C1=O'], task_names: ['measured log solubility in mols per litre']>


In [ ]:
print(test_dataset)

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 ' ...
 'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43' 'Cc1occc1C(=O)Nc2ccccc2'
 'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O '], task_names: ['measured log solubility in mols per litre']>


In [ ]:
model = dc.models.MPNNModel(n_tasks=len(tasks),n_atom_feat = 75, n_pair_feat = 14, T = 4,
                            M = 5, batch_size =10,epochs = 50,dropout = 0.5, learning_rate = 0.001, mode ='regression')

In [ ]:
model.fit(train_dataset, nb_epoch=100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/message_passing/edge_network/Reshape_4:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/message_passing/edge_network/Reshape_3:0", shape=(None, 100), dtype=float32), dense_shape=Tensor("gradient_tape/model/message_passing/edge_network/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/message_passing/edge_network/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/message_passing/edge_network/Reshape_8:0", shape=(None, 100), dtype=float32), dense_shape=T

0.035175209045410154

In [ ]:
predicted = model.predict_on_batch(test_dataset.X)

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))
mae = dc.metrics.mean_absolute_error(y_true=test_dataset.y,y_pred=predicted)
print("Mean Absolute Error:", mae)

Training set score: {'pearson_r2_score': 0.971488231127746}
Test set score: {'pearson_r2_score': 0.8360226973093162}
Mean Absolute Error: 0.38812054139040525


In [ ]:
check_data = open('Coumarin_solub.csv','r')
pred_featurizer = dc.feat.WeaveFeaturizer()
pred_task = []
pred_loader = dc.data.CSVLoader(tasks=pred_task, feature_field='SMILES', featurizer=pred_featurizer)
pred_dataset = pred_loader.featurize(check_data)

/usr/local/lib/python3.7/dist-packages/deepchem/data/data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)


In [ ]:
print(pred_dataset)

<DiskDataset X.shape: (1748,), y.shape: (), w.shape: (), task_names: []>


In [ ]:
solubilities = model.predict_on_batch(pred_dataset.X)
for molecule, solubility, in zip(test_dataset.ids, solubilities):
   print(solubility, molecule)

[0.57252777] c1cc2ccc3cccc4ccc(c1)c2c34
[0.2851803] Cc1cc(=O)[nH]c(=S)[nH]1
[0.45169103] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[0.6810143] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-0.05981854] C1=Cc2cccc3cccc1c23
[0.69507796] CC1CO1
[0.06639539] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[0.35141703] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[0.84595454] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[0.8401174] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 
[0.22434755] COc2c1occc1cc3ccc(=O)oc23 
[-0.46284434] CN2C(=C(O)c1ccccc1S2(=O)=O)C(=O)Nc3ccccn3 
[-0.12341562] Cc3cc2nc1c(=O)[nH]c(=O)nc1n(CC(O)C(O)C(O)CO)c2cc3C
[0.87544847] c1ccc(cc1)c2ccc(cc2)c3ccccc3
[0.07216039] CC34CC(=O)C1C(CCC2=CC(=O)CCC12C)C3CCC4(=O) 
[-0.38855216] c1ccc2c(c1)sc3ccccc23
[0.06315508] CC23Cc1cnoc1C=C2CCC4C3CCC5(C)C4CCC5(O)C#C
[-0.66800094] OC(C(=O)c1ccccc1)c2ccccc2
[0.10913825] OCC2OC(Oc1ccccc1CO)C(O)C(O)C2O
[0.27597663] CC3C2CCC1(C)C=CC(=O)C(=C1C2OC3=O)C
[0.11993491] O=Cc2ccc1OCOc1c2 
[-0.32108572] CC1CCCCC1NC(=O)Nc2ccccc2
[0.1